### Desafio Codenation - nono módulo    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                                                 by Flávio

# Descubra quem fez a prova do Enem 2016 apenas para treino



In [1]:
import pandas as pd
import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

**Vamos ler os dataframes fornecidos**

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#### As colunas do dataframe de teste fornecem a pista para os parâmetros a serem avaliados como mais importantes para a classificação

In [3]:
all_features = df_test.columns.to_list()
all_features.append('IN_TREINEIRO')
all_features

['NU_INSCRICAO',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'TP_DEPENDENCIA_ADM_ESC',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'IN_IDOSO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 'Q027',
 'Q047',
 'IN_TREINEIRO']

In [4]:
# ajustando o df treino para as colunas disponíveis
df_train = df_train[all_features]


**Achei este [profiling](https://minerandodados.com.br/analise-de-dados-com-python-usando-pandas/) uma excelente dica do pessoal do Minerando Dados**

In [5]:
pandas_profiling.ProfileReport(df_train)

C:\ProgramData\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,44
Number of observations,13730
Total Missing (%),8.1%
Total size in memory,4.6 MiB
Average record size in memory,352.0 B
Numeric,18
Categorical,10
Boolean,9
Date,0
Text (Unique),1
Rejected,6


In [6]:
# Analisando o resultado do profiling e correlações apresentadas vamos manter apenas algumas colunas
Y_train = df_train['IN_TREINEIRO']

cols_to_keep = ['NU_INSCRICAO', 'NU_IDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU']
df_train = df_train[cols_to_keep]

In [7]:
df_train.isna().mean()

NU_INSCRICAO       0.0
NU_IDADE           0.0
TP_ST_CONCLUSAO    0.0
TP_ANO_CONCLUIU    0.0
dtype: float64

In [8]:
# adaptando o df test
df_test = df_test[cols_to_keep]


In [9]:
# vejamos quão desbalanceado está nossa amostra para classificação...

Y_train.value_counts()

0    11947
1     1783
Name: IN_TREINEIRO, dtype: int64

### Iremos analisar a performance de três classificadores a partir da curva ROC ou AUC ROC

**Recomento dar uma olhad neste [artigo](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/) e também nessa [live](https://www.youtube.com/watch?v=860F540KmkU) do Mario Filho**

In [10]:
# como exercicio vale a pena modificar a proporção teste/treino e ver a variação em performance dos modelos...
x_train, x_test, y_train, y_test = train_test_split(df_train, Y_train, test_size=0.2, random_state = 2020)


### Regressão Logistica

In [11]:
logistic = LogisticRegression()
logistic.fit(x_train.iloc[:, 1:], y_train)

lr_probs = logistic.predict_proba(x_test.iloc[:, 1:])
lr_probs = lr_probs[:, 1]

#AUC ROC
lr_auc = roc_auc_score(y_test, lr_probs)
print('Logistic: ROC AUC=%.6f' % (lr_auc))

Logistic: ROC AUC=0.995619


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### K-nearest neighbors

In [12]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x_train.iloc[:, 1:], y_train)

knn_probs = knn.predict_proba(x_test.iloc[:, 1:])
knn_probs = knn_probs[:, 1]

#AUC ROC
knn_auc = roc_auc_score(y_test, knn_probs)
print('k-nearest: ROC AUC=%.6f' % (knn_auc))

k-nearest: ROC AUC=0.999871


### Random Forest

In [13]:
random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(x_train.iloc[:, 1:], y_train)

rf_probs = random_forest.predict_proba(x_test.iloc[:, 1:])
rf_probs = rf_probs[:, 1]

#AUC ROC
rf_auc = roc_auc_score(y_test, rf_probs)
print('Random Forest: ROC AUC=%.6f' % (rf_auc))

Random Forest: ROC AUC=0.999891


**Visto que o RF apresentou o melhor resultado dada a métrica escolhida, lá vamos nós...**

In [14]:
# usando todos os dados de treino disponíveis
random_forest.fit(df_train.iloc[:, 1:], Y_train)
rf_labeled = random_forest.predict(df_test.iloc[:, 1:])

df_rf_labeled = pd.DataFrame(rf_labeled, columns=['IN_TREINEIRO']) 

In [15]:
# Random Forest - 
df_answer = pd.concat([df_test, df_rf_labeled], axis=1)

In [16]:
cols_to_save = ['NU_INSCRICAO', 'IN_TREINEIRO']

df_answer.to_csv('answer.csv', columns=cols_to_save, index=False)

In [17]:
# final check
df=pd.read_csv('answer.csv')
df.head()

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0


### Score final de 99.737 %